In [171]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

#Add math and reading Pass (True), Fail (False) flags to student data
new_student_data = student_data
new_student_data['math_pass'] = new_student_data['math_score'].apply(lambda x: (x > 69))
new_student_data['reading_pass'] = new_student_data['reading_score'].apply(lambda x: (x > 69))
new_student_data['both_pass'] = new_student_data['math_pass'] & new_student_data['reading_pass']

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [172]:
print(school_data.head())
print(new_student_data.head())
print(school_data_complete.head())


School ID            school_name      type  size   budget
0          0      Huang High School  District  2917  1910635
1          1   Figueroa High School  District  2949  1884411
2          2    Shelton High School   Charter  1761  1056600
3          3  Hernandez High School  District  4635  3022020
4          4    Griffin High School   Charter  1468   917500
   Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  math_pass  reading_pass  both_pass  
0             66          79       True         False      False  
1             94          61      False          True      False  
2             90          6

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [173]:
tot_num_schools = school_data['school_name'].nunique()
tot_num_students = new_student_data['Student ID'].nunique()
tot_budget = school_data['budget'].sum()
avg_math_score = new_student_data['math_score'].sum()/tot_num_students
avg_reading_score = new_student_data['reading_score'].sum()/tot_num_students
math_passing_students = new_student_data.loc[student_data['math_score']>69]
reading_passing_students = new_student_data.loc[new_student_data['reading_score']>69]
both_passing_students = pd.merge(math_passing_students, reading_passing_students, on='Student ID')
math_passing_pct = new_student_data['math_pass'].sum()/tot_num_students
reading_passing_pct = new_student_data['reading_pass'].sum()/tot_num_students
both_passing_pct = new_student_data['both_pass'].sum()/tot_num_students


district_summary_df = pd.DataFrame({"Total Schools":[tot_num_schools], "Total Students":[tot_num_students], "Total Budget":[tot_budget], "Average Math Score":[avg_math_score], "Average Reading Score":[avg_reading_score], "Passing Math":[math_passing_pct], "Passing Reading":[reading_passing_pct], "Overall Passing":[both_passing_pct]})

district_summary_df.style.hide_index().format({"Total Budget": "${:,.0f}", "Passing Math": "{:.2%}", "Passing Reading": "{:.2%}", "Overall Passing": "{:.2%}", "Total Schools": "{:,.0f}", "Total Students": "{:,.0f}", "Average Math Score": "{:,.2f}", "Average Reading Score": "{:,.2f}"})

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing
15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [174]:
school_grouped = new_student_data.groupby(['school_name'])
school_summary_df = school_data[['school_name', 'type', 'size', 'budget']]
school_summary_df = school_summary_df.rename(columns={'school_name': 'School Name', 'type':'School Type', 'size':'Total Students', 'budget':'Total School Budget'})

school_summary_df['Per Student Budget']=school_summary_df['Total School Budget']/school_summary_df['Total Students']
school_summary_df = school_summary_df.set_index('School Name')
school_summary_df = school_summary_df.sort_index(axis = 0)

school_avg_math_score = school_grouped['math_score'].sum()/school_grouped['Student ID'].nunique()
school_avg_reading_score = school_grouped['reading_score'].sum()/school_grouped['Student ID'].nunique()
school_math_passing_pct = school_grouped['math_pass'].sum()/school_grouped.size()
school_reading_passing_pct = school_grouped['reading_pass'].sum()/school_grouped.size()
school_both_passing_pct = school_grouped['both_pass'].sum()/school_grouped.size()

school_summary_df['Average Math Score'] = school_avg_math_score
school_summary_df['Average Reading Score'] = school_avg_reading_score
school_summary_df['Passing Math'] = school_math_passing_pct
school_summary_df['Passing Reading'] = school_reading_passing_pct
school_summary_df['Overall Passing'] = school_both_passing_pct

school_summary_df.style.format({"Total School Budget": "${:,.0f}", "Per Student Budget": "${:,.2f}", "Passing Math": "{:.2%}", "Passing Reading": "{:.2%}", "Overall Passing": "{:.2%}", "Total Students": "{:,.0f}", "Average Math Score": "{:,.2f}", "Average Reading Score": "{:,.2f}"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [175]:
school_summary_df.sort_values(by=['Overall Passing'], ascending=False).head().style.format({"Total School Budget": "${:,.0f}", "Per Student Budget": "${:,.2f}", "Passing Math": "{:.2%}", "Passing Reading": "{:.2%}", "Overall Passing": "{:.2%}", "Total Students": "{:,.0f}", "Average Math Score": "{:,.2f}", "Average Reading Score": "{:,.2f}"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [176]:
school_summary_df.sort_values(by=['Overall Passing'], ascending=True).head().style.format({"Total School Budget": "${:,.0f}", "Per Student Budget": "${:,.2f}", "Passing Math": "{:.2%}", "Passing Reading": "{:.2%}", "Overall Passing": "{:.2%}", "Total Students": "{:,.0f}", "Average Math Score": "{:,.2f}", "Average Reading Score": "{:,.2f}"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [298]:
#by_school_by_grade = pd.DataFrame(new_student_data.groupby(['grade', 'school_name'])['math_score'].mean())
#by_school_by_grade_avg_math=by_school_by_grade['math_score'].sum()/by_school_by_grade['Student ID'].count()
#grade_classes_list = new_student_data['grade'].unique()
#school_names_list = new_student_data['school_name'].unique()
#math_scores_by_grade_df=pd.DataFrame([], columns=grade_classes_list, index=school_names_list).sort_index()
#math_scores_by_grade_df = math_scores_by_grade_df.reindex(sorted(math_scores_by_grade_df.columns), axis=1)
#math_scores_by_grade_df = math_scores_by_grade_df[ ['9th'] + [ col for col in math_scores_by_grade_df.columns if col != '9th' ] ]

bsbg_math_df = new_student_data.pivot_table(values='math_score', index='school_name',columns='grade', aggfunc=np.mean)
bsbg_math_df=bsbg_math_df[ ['9th'] + [ col for col in bsbg_math_df.columns if col != '9th' ] ]

bsbg_math_df.style.format("{:,.2f}")

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [299]:
bsbg_reading_df = new_student_data.pivot_table(values='reading_score', index='school_name',columns='grade', aggfunc=np.mean)
bsbg_reading_df=bsbg_reading_df[ ['9th'] + [ col for col in bsbg_reading_df.columns if col != '9th' ] ]

bsbg_reading_df.style.format("{:,.2f}")

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)